In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# read the data, we are using the training csv from Kaggle Digit Recognizer 
# https://www.kaggle.com/c/digit-recognizer/data
data = pd.read_csv('train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# using numpy arrays to apply linear algebra
data = np.array(data)
r, c = data.shape
np.random.shuffle(data)

# splitting into train and val sets
data_val = data[0:1000].T
Y_val = data_val[0]
X_dev = data_val[1:c]
X_dev = X_dev / 255.

data_train = data[1000:r].T
Y_train = data_train[0]
X_train = data_train[1:c]
X_train = X_train / 255.


In [5]:
# initialise all the parameters we need
def init_params():
    W1 = np.random.randn(10, 784)
    b1 = np.random.randn(10, 1)

    W2 = np.random.randn(10, 10)
    b2 = np.random.randn(10, 1)
    
    return W1, b1, W2, b2

In [6]:
# predictions and accuracy functions
def get_preds(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [7]:
# we are going to use the relu and softmax functions, let's define them:
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

# derivative of relu
def der_ReLU(Z):
    return Z > 0

In [8]:
# one hot encoding function
def oh_encoder(Y):
    oh_Y = np.zeros((Y.size, Y.max() + 1))
    oh_Y[np.arange(Y.size), Y] = 1
    oh_Y = oh_Y.T
    return oh_Y

In [9]:
# define de forward pass
def forward(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)

    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2

In [10]:
# define the backpropagation function
def back_propagation(Z1, A1, Z2, A2, W2, X, Y):
    r = Y.size
    oh_Y = oh_encoder(Y)

    dZ2 = A2 - oh_Y
    dW2 = 1 /  r * dZ2.dot(A1.T)
    db2 = 1 / r * np.sum(dZ2)

    dZ1 = W2.T.dot(dZ2) * der_ReLU(Z1)
    dW1 = 1 /  r * dZ1.dot(X.T)
    db1 = 1 / r * np.sum(dZ1)

    return dW1, db1, dW2, db2

In [11]:
# define the update parameters function
def upd_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [14]:
# define the gradient descent method
def grad_descent(X, Y, iter, alpha):
    W1, b1, W2, b2 = init_params()

    for i in range(iter):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = upd_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 50 == 0:
            print('Iteration: ', i)
            print('Accuracy: ', get_accuracy(get_preds(A2), Y))

    return W1, b1, W2, b2

In [17]:
W1, b1, W2, b2 = grad_descent(X_train, Y_train, 1000, 0.1)

Iteration:  0
Accuracy:  0.1552439024390244
Iteration:  50
Accuracy:  0.2658048780487805
Iteration:  100
Accuracy:  0.2870243902439024
Iteration:  150
Accuracy:  0.31485365853658537
Iteration:  200
Accuracy:  0.3404390243902439
Iteration:  250
Accuracy:  0.36317073170731706
Iteration:  300
Accuracy:  0.38058536585365854
Iteration:  350
Accuracy:  0.39702439024390246
Iteration:  400
Accuracy:  0.38890243902439026
Iteration:  450
Accuracy:  0.39846341463414636
Iteration:  500
Accuracy:  0.4059756097560976
Iteration:  550
Accuracy:  0.4130243902439024
Iteration:  600
Accuracy:  0.41797560975609754
Iteration:  650
Accuracy:  0.42509756097560975
Iteration:  700
Accuracy:  0.4323170731707317
Iteration:  750
Accuracy:  0.4389268292682927
Iteration:  800
Accuracy:  0.4457560975609756
Iteration:  850
Accuracy:  0.4527560975609756
Iteration:  900
Accuracy:  0.46207317073170734
Iteration:  950
Accuracy:  0.4712682926829268
